# Cluster Galaxy Annotation Tool (Colab)

This notebook lets you **click on an image** to label:
- **Galaxy members** (cluster members) – red markers
- **Background objects** (cluster non-members) – blue markers

Use the buttons below the plot to switch modes. When done, run the last cell to download your results.

## 1. Setup and upload your image

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import io
import csv
import re
import os
from datetime import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets

print("Libraries loaded.")

: 

In [ ]:
# Upload your PNG image (or use any image path if you have one in Colab)
uploaded = files.upload()
image_filename = list(uploaded.keys())[0]
print(f"Uploaded: {image_filename}")

## 2. Click to annotate

1. **Run the cell below** — the image will appear; **click on it** to add points (red = members, blue = background).
2. **Switch mode** with the buttons before each click.
3. When finished, run the "Save results" cell to download.

In [ ]:
# No extra packages needed: image clicks use Colab's HTML + JavaScript + register_callback

In [ ]:
from PIL import ImageDraw
import io
import base64
from IPython.display import display, HTML, Javascript

# Load image
img = Image.open(image_filename).convert("RGB")
h, w = img.size[1], img.size[0]

# Storage
members_coords = []
non_members_coords = []
current_mode = "members"
marker_r = 4

# Display size (max 600px)
max_side = 600
if w >= h:
    display_w, display_h = max_side, int(h * max_side / w)
else:
    display_h, display_w = max_side, int(w * max_side / h)

def image_to_png_bytes(image_pil, members_list, non_members_list):
    out = image_pil.copy()
    draw = ImageDraw.Draw(out)
    for (x, y) in members_list:
        draw.ellipse([x - marker_r, y - marker_r, x + marker_r, y + marker_r],
                     fill=(255, 0, 0), outline=(200, 0, 0))
    for (x, y) in non_members_list:
        draw.ellipse([x - marker_r, y - marker_r, x + marker_r, y + marker_r],
                     fill=(0, 0, 255), outline=(0, 0, 200))
    buf = io.BytesIO()
    out.save(buf, format="PNG")
    return buf.getvalue()

def add_point(x, y):
    global members_coords, non_members_coords, current_mode
    x, y = int(x), int(y)
    x = max(0, min(x, w - 1))
    y = max(0, min(y, h - 1))
    if current_mode == "members":
        members_coords.append((x, y))
        print(f"Member #{len(members_coords)}: ({x}, {y})")
    else:
        non_members_coords.append((x, y))
        print(f"Background #{len(non_members_coords)}: ({x}, {y})")
    new_bytes = image_to_png_bytes(img, members_coords, non_members_coords)
    b64 = base64.b64encode(new_bytes).decode("ascii")
    display(Javascript('document.getElementById("cluster-anno-img").src = "data:image/png;base64,' + b64 + '";'))

from google.colab import output
output.register_callback("add_point", add_point)

def set_mode_members(_):
    global current_mode
    current_mode = "members"
    status.value = "Mode: Galaxy Members (red)"

def set_mode_non_members(_):
    global current_mode
    current_mode = "non_members"
    status.value = "Mode: Background Objects (blue)"

status = widgets.Label(value="Mode: Galaxy Members (red)")
btn_members = widgets.Button(description="Mark Galaxy Members (M)", style=dict(button_color='lightcoral'))
btn_non = widgets.Button(description="Mark Background Objects (B)", style=dict(button_color='lightblue'))
btn_members.on_click(set_mode_members)
btn_non.on_click(set_mode_non_members)

# Buttons and label first, then image below
display(widgets.VBox([
    widgets.HBox([status, btn_members, btn_non]),
    widgets.Label(value="Click on the image below to add points (red = members, blue = background):"),
]))

# Initial image as base64
img_bytes = image_to_png_bytes(img, members_coords, non_members_coords)
img_b64 = base64.b64encode(img_bytes).decode("ascii")

# Image as HTML so we can attach a JS click handler (works in Colab)
display(HTML(
    '<img id="cluster-anno-img" src="data:image/png;base64,' + img_b64 + '" '
    'width="' + str(display_w) + '" height="' + str(display_h) + '" '
    'style="cursor:crosshair; border:1px solid #ccc;">'
))
display(Javascript("""
(function() {
  var img = document.getElementById("cluster-anno-img");
  if (!img) return;
  var w = %d, h = %d, dw = %d, dh = %d;
  img.onclick = async function(e) {
    var ox = e.offsetX, oy = e.offsetY;
    var x = Math.round(ox * w / dw);
    var y = Math.round(oy * h / dh);
    x = Math.max(0, Math.min(x, w - 1));
    y = Math.max(0, Math.min(y, h - 1));
    await google.colab.kernel.invokeFunction("add_point", [x, y], {});
  };
})();
""" % (w, h, display_w, display_h)))

## 3. Save results and download

Run this cell when you are done clicking. It will:
- Create a `cluster_001` folder (or next number)
- Save `image.png`, `cluster_members.csv`, `cluster_non_members.csv`
- Save `image_marked_members.png` and `image_marked_non_members.png`
- Zip the folder, upload a copy to the research Google Drive folder, and trigger a download to your computer.

In [ ]:
import zipfile

# Choose output folder name (cluster_001, cluster_002, ...)
base_dir = "/content"
existing = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and re.match(r'^cluster_\\d{3}$', d)]
nums = [int(re.search(r'\\d{3}$', d).group()) for d in existing if re.search(r'\\d{3}$', d)]
next_num = max(nums, default=0) + 1
folder_name = f"cluster_{next_num:03d}"
out_dir = os.path.join(base_dir, folder_name)
os.makedirs(out_dir, exist_ok=True)
print(f"Output folder: {out_dir}")

# Copy original image
img.save(os.path.join(out_dir, "image.png"))

# Save CSVs
with open(os.path.join(out_dir, "cluster_members.csv"), "w", newline='') as f:
    w = csv.writer(f)
    w.writerow(["X", "Y"])
    w.writerows(members_coords)
with open(os.path.join(out_dir, "cluster_non_members.csv"), "w", newline='') as f:
    w = csv.writer(f)
    w.writerow(["X", "Y"])
    w.writerows(non_members_coords)

# Draw marked images
marker_r = 2
def draw_markers(coords, color_rgb, path):
    out_img = img.copy()
    draw = ImageDraw.Draw(out_img)
    for x, y in coords:
        draw.ellipse([x - marker_r, y - marker_r, x + marker_r, y + marker_r], fill=color_rgb, outline=color_rgb)
    out_img.save(path)

if members_coords:
    draw_markers(members_coords, (255, 0, 0), os.path.join(out_dir, "image_marked_members.png"))
if non_members_coords:
    draw_markers(non_members_coords, (0, 0, 255), os.path.join(out_dir, "image_marked_non_members.png"))

zip_path = os.path.join(base_dir, f"{folder_name}.zip")
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for root, dirs, files in os.walk(out_dir):
        for f in files:
            fp = os.path.join(root, f)
            zf.write(fp, os.path.relpath(fp, base_dir))

print(f"Members: {len(members_coords)}, Non-members: {len(non_members_coords)}")

# Upload a copy to the researcher's Google Drive folder (user will be asked to sign in and allow access once)
RESEARCH_DRIVE_FOLDER_ID = "1ULmbBoMlImUkmUYzn0NEgZak5CtUHUjz"
if RESEARCH_DRIVE_FOLDER_ID:
    try:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=False)
        from google.colab import auth
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaFileUpload
        import uuid
        auth.authenticate_user()
        service = build("drive", "v3", cache_discovery=False)
        unique_name = f"{folder_name}_{uuid.uuid4().hex[:8]}.zip"
        file_meta = {"name": unique_name, "parents": [RESEARCH_DRIVE_FOLDER_ID]}
        media = MediaFileUpload(zip_path, mimetype="application/zip", resumable=True)
        service.files().create(body=file_meta, media_body=media, fields="id").execute()
        print("Results uploaded to research folder. Thank you!")
    except Exception as e:
        print("Upload to research folder failed (you can still use your download):", e)

print("Downloading zip...")
#files.download(zip_path)